# Safety in Funflow

Besides the safety that comes with immutable data and strong, static typing in Haskell, Funflow considers common properties of data workflow/pipeline authorship and strives to offer some extra safety tailored to this domain. In particular, Funflow offers some ___configuration-time_ validation__.

What does this mean? Well, one upside of Funflow's ability to decouple pipeline _declaration_ from pipeline _execution_ is that we gain an additional layer of information, and a platform/opportunity to perform more informed validation of what will unfold at execution time. As such, we can consider the possibility for errors that could arise at arbitrary timepoints in the course of execution of a workflow.

Just as static typing allows for _compile-time_ checking, catching errors that could otherwise occur much later in a program's lifecycle (during _run-time_), this concrete decoupling of conceptual phases of a pipeline's lifecycle allows us to surface errors. We'd much rather know about and be able to fix an error before a pipeline ever starts than to run into the error after great cost (certainly of compute time, and perhaps also of compute money!)

In [51]:
import Funflow (ioFlow, pureFlow)
import Funflow.Config ( readEnv, ConfigKey, Configurable, Configurable( ConfigFromEnv ) )
import Funflow.Tasks.Simple (SimpleTask (IOTask))
import System.Environment (lookupEnv, setEnv, unsetEnv)

In [11]:
lookupEnv "NONEXISTENT"

Nothing

In [13]:
setEnv "NONEXISTENT" "hello"

In [14]:
lookupEnv "NONEXISTENT"

Just "hello"

In [7]:
import qualified Data.Text as T
cfgpar :: Configurable String
cfgpar = ConfigFromEnv (T.pack "NONEXISTENT")

In [83]:
:t cfgpar

cfgpar :: Configurable String

In [43]:
let myFun = readEnv

In [44]:
:t myFun

myFun :: forall (m :: * -> *). MonadIO m => ConfigKey -> m (HashMap Text String)

In [48]:
:t (lookupEnv . T.unpack)

(lookupEnv . T.unpack) :: Text -> IO (Maybe String)

In [108]:
import Funflow (toFlow, Flow)
import Funflow.Tasks.Simple ( SimpleTask (PureTask) )
import Funflow.Config (getConfigKey)

unsetEnv "NONEXISTENT"

flow :: Flow i (Maybe ConfigKey)
flow = toFlow $ PureTask (\_ -> getConfigKey cfgpar)

In [79]:
import Funflow (runFlow)

In [97]:
:t fmap T.unpack . getConfigKey

fmap T.unpack . getConfigKey :: forall a. Configurable a -> Maybe String

In [109]:
-- TODO: how to get this to fail at CONFIG-time? (since the Configurable isn't defined)
runFlow flow cfgpar :: IO (Maybe ConfigKey)

Just "NONEXISTENT"

In [22]:
import System.Environment (lookupEnv)
lookupEnv "FUNFLOW_TEST"

Nothing

In [23]:
import qualified Funflow.Tasks.Docker as DE
import Funflow.Config (Configurable(ConfigFromEnv))
dockConf = DE.DockerTaskConfig {DE.image = T.pack "bash:latest", DE.command = T.pack "echo", DE.args = [DE.Arg $ ConfigFromEnv (T.pack "FUNFLOW_TEST")]}
:t dockConf

dockConf :: DockerTaskConfig

In [32]:
import Funflow.Flow (dockerFlow, Flow)
import qualified Data.CAS.ContentStore as CS
dockFlow :: Flow DE.DockerTaskInput CS.Item
dockFlow = dockerFlow dockConf
:t dockFlow

dockFlow :: forall (r :: RopeRec) (core :: BinEff) (m :: * -> *) (mantle :: [Strand]).
(RecElemFCtx r (Weaver core), MonadIO m, RecElem r ("docker" ::: DockerTask) ("docker" ::: DockerTask) mantle mantle (RIndex ("docker" ::: DockerTask) mantle),
 RecElem r ("simple" ::: SimpleTask) ("simple" ::: SimpleTask) mantle mantle (RIndex ("simple" ::: SimpleTask) mantle),
 RecElem r ("store" ::: StoreTask) ("store" ::: StoreTask) mantle mantle (RIndex ("store" ::: StoreTask) mantle), ArrowChoice core, ThrowEffect SomeException core, ThrowEffect StringException core,
 ThrowEffect DockerClientError core, TryEffect SomeException core, TryEffect StringException core, TryEffect DockerClientError core, SieveTrans m core, ProvidesCaching core,
 FieldType "simple" mantle ~ SimpleTask, FieldType "store" mantle ~ StoreTask, FieldType "docker" mantle ~ DockerTask) =>
Rope r mantle core DockerTaskInput Item

In [33]:
DE.DockerTaskInput

: 